In [1]:
import pickle
from operator import itemgetter
import pandas as pd
import plotly.graph_objects as go

from hydra.plotly_charts import PlotlyPriceChart, supersample_data
from hydra.utils import split_dupes


SYMBOLS = ["circle", "square", "diamond-tall", "diamond-wide"]
COLORS = 'afmhot_r'
filename = "../2020-05-01 1d count=365 overlap=18h detrend=True.enviro"


file = None
with open(filename, "rb") as handle:
    file = pickle.load(handle)
interesting, charts, aggregate, inputs = itemgetter('data', 'charts', 'aggregate', 'inputs')(file)
timecap = pd.to_timedelta(inputs['timecap'])
overlap = pd.to_timedelta(inputs['overlap'])
traceCount = int(1.0/((timecap-overlap)/timecap))
complexity = {'date':[], 'values':[]}
for idx, (colname, frequencies) in enumerate(aggregate.iteritems()):
    complexity['date'].append(charts[idx]['trendline'].index[0])
    complexity['values'].append((frequencies != 0).sum())


pair = inputs['pair']
startDate = charts[0]['startDate']
endDate = charts[-1]['endDate']
# startDate = "2017-05-15 00:00"
# endDate = "2020-05-16 00:00"
print(startDate, endDate)
chart_data = {i:{
    "trendline":[],
    "ifft_price":[]
} for i in range(traceCount)}

for index, chart in enumerate(charts):
    idx = index % traceCount
    chart_data[idx]['trendline'].append(chart['trendline'])
    chart_data[idx]['ifft_price'].append(chart['majority_price'])
    

# print(trendlines)
priceChart = PlotlyPriceChart(pair, startDate, endDate)

def add_trace(priceChart, data, chartParams={}, interval=1):
    d = pd.concat(data)
    supersampled = supersample_data(d, interval)
    priceChart.add_trace( 
        go.Scatter(x=supersampled[1440].index, y=supersampled[1440].values, **chartParams), 
        supersampled, 
        {"x": "index", "y": "values"},
    )
    
# print(len(aggregate))
for index in range(traceCount):
    add_trace(priceChart, chart_data[index]['ifft_price'], {"mode":"lines", "name":f"ifft-{index}"})
for index in range(traceCount):
    add_trace(priceChart, chart_data[index]['trendline'], {"mode":"markers", "name":f"trend-{index}", "marker_symbol":SYMBOLS[index]})

priceChart.add_trace(
    go.Scatter(x=complexity['date'], y=complexity['values']), loc=(1,1), log=True
)
    # priceChart.figure.add_vline(x=startDate, line_width=3)

priceChart.render()


2020-05-01 00:00:00 2020-08-01 00:00:00
add_trace Scatter({
    'x': [2020-05-01 00:00:00, 2020-05-01 06:00:00, 2020-05-01 12:00:00, ...,
          2020-07-30 12:00:00, 2020-07-30 18:00:00, 2020-07-31 00:00:00],
    'y': [6, 5, 2, ..., 1, 2, 2]
}) None None (1, 1)


FigureWidget({
    'data': [{'close': array([ 9004.7    ,  8824.6    ,  8854.9    ,  8983.1    ,  9052.6    ,
…